<a href="https://www.kaggle.com/code/coderpython/saransh-rajpurohit-celebal-project?scriptVersionId=139415497" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import os
import glob
import random
import numpy as np
import pandas as pd

import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

from tqdm import tqdm

from PIL import Image

from tensorflow.keras.utils import to_categorical

import seaborn as sns
import matplotlib.image as img
import matplotlib.pyplot as plt

In [ ]:
train_csv = pd.read_csv("../input/human-action-recognition-har-dataset/Human Action Recognition/Training_set.csv")
test_csv = pd.read_csv("../input/human-action-recognition-har-dataset/Human Action Recognition/Testing_set.csv")

In [ ]:
train_fol = glob.glob("../input/human-action-recognition-har-dataset/Human Action Recognition/train/*") 
test_fol = glob.glob("../input/human-action-recognition-har-dataset/Human Action Recognition/test/*")

In [ ]:
train_csv.label.value_counts()

In [ ]:
train_csv

In [ ]:
filename = train_csv['filename']
situation = train_csv['label']

In [ ]:
imgg = "Image_{}.jpg".format(1)
train = "../input/human-action-recognition-har-dataset/Human Action Recognition/train/"
testImage = img.imread(train + imgg)
plt.imshow(testImage)
plt.title("{}".format(train_csv.loc[train_csv['filename'] == "{}".format(imgg), 'label'].item()))
plt.show()

#  **Pre Processing**

In [ ]:
img_data = []
img_label = []
length = len(train_fol)
for i in (range(len(train_fol)-1)):
    t = '../input/human-action-recognition-har-dataset/Human Action Recognition/train/' + filename[i]    
    temp_img = Image.open(t)
    img_data.append(np.asarray(temp_img.resize((160,160))))
    img_label.append(situation[i])

In [ ]:
inp_shape = (160, 160,3)

In [ ]:
arr = img_data
arr = np.asarray(arr)
type(arr)

In [ ]:
y_train = to_categorical(np.asarray(train_csv['label'].factorize()[0]))
print(y_train[0])

In [ ]:
vgg_model = Sequential()

pretrained_model= tf.keras.applications.VGG16(include_top=False,
                   input_shape=(160,160,3),
                   pooling='avg',classes=15,
                   weights='imagenet')

for layer in pretrained_model.layers:
        layer.trainable=False

vgg_model.add(pretrained_model)
vgg_model.add(Flatten())
vgg_model.add(Dense(512, activation='relu'))
vgg_model.add(Dense(15, activation='softmax'))

In [ ]:
vgg_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
vgg_model.summary()

In [ ]:
history = vgg_model.fit(arr,y_train, epochs=40)

In [ ]:
vgg_model.save_weights("model.h5")

In [ ]:
accurate = history.history['accuracy']
plt.plot(accurate)

# **Inference Pipeline**

In [ ]:
def test_predict(test_image):
    situation=["sitting","using_laptop","hugging","sleeping","drinking",
           "clapping","dancing","cycling","calling","laughing"
          ,"eating","fighting","listening_to_music","running","texting"]
    image = Image.open(test_image)
    input_img = np.asarray(image.resize((160,160)))
    result = vgg_model.predict(np.asarray([input_img]))

    itemindex = np.where(result==np.max(result))
    prediction = itemindex[1][0]
    print("probability: "+str(np.max(result)*100) + "%\nPredicted class : ", situation[prediction])

    image = img.imread(test_image)
    plt.imshow(image)
 

In [ ]:
test_predict('../input/human-action-recognition-har-dataset/Human Action Recognition/test/Image_50.jpg')